# 로컬 개발 코드
- 로컬에서 주피터 노트북(Jupyter Notebook), 주피터 랩(JupyterLab) 또는 파이썬(Python)을 이용한다. 
- 사이킷 런(scikit-learn), 텐서플로우(tensorflow), 파이토치(pytorch)를 사용하여 딥러닝 프로그램을 개발한다.
- 파일명: 0_local_iris_classification.ipynb

### 로컬 개발 워크플로우(workflow)  
- 로컬 개발 워크플로우를 다음의 4단계로 분리한다.

1. **데이터 세트 준비(Data Setup)**
- 로컬 저장소에서 전처리 및 학습에 필요한 학습 데이터 세트를 준비한다.

2. **데이터 전처리(Data Preprocessing)**
- 데이터 세트의 분석 및 정규화(Normalization)등의 전처리를 수행한다.
- 데이터를 모델 학습에 사용할 수 있도록 가공한다.
- 추론과정에서 필요한 경우, 데이터 전처리에 사용된 객체를 meta_data 폴더 아래에 저장한다.

3. **학습 모델 훈련(Train Model)**
- 데이터를 훈련에 사용할 수 있도록 가공한 뒤에 학습 모델을 구성한다. 
- 학습 모델을 준비된 데이터 세트로 훈련시킨다.
- 정확도(Accuracy)나 손실(Loss)등 학습 모델의 성능을 검증한다.
- 학습 모델의 성능 검증 후, 학습 모델을 배포한다.
- 배포할 학습 모델을 meta_data 폴더 아래에 저장한다.

4. **추론(Inference)**
- 저장된 전처리 객체나 학습 모델 객체를 준비한다.
- 추론에 필요한 테스트 데이터 세트를 준비한다.
- 배포된 학습 모델을 통해 테스트 데이터에 대한 추론을 진행한다. 

# 아이리스 분류(Iris Classification)

### Pytorch를 이용한 아이리스 품종 분류 워크플로우(workflow)
- 아이리스 데이터 파일(iris.csv) 얻기
- 학습을 위한 데이터 전처리(훈련 데이터, 테스트 데이터)
- SVM을 이용한 모델 생성 및 훈련
- 아이리스 품종 추론

### 아이리스 데이터 세트
- 아이리스 데이터 세트에는 150개의 데이터 세트가 저장되어 있다.
- 아이리스 데이터 세트는 꽃받침 길이, 꽃받침 폭, 꽃잎 길이, 꽃잎 폭 총 5개의 컬럼으로 구성되어 있다.


In [1]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

c:\Users\USER\anaconda3\envs\T3Q_DL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **1. 데이터 세트 준비(Data Setup)**

In [2]:
# IRIS dataset를 불러온다.
dataset = pd.read_csv('dataset/iris.csv')

# Species을 numerics로 변환한다.
dataset.loc[dataset.iloc[:,-1]=='Iris-setosa', dataset.columns[-1]] = 0
dataset.loc[dataset.iloc[:,-1]=='Iris-versicolor', dataset.columns[-1]] = 1
dataset.loc[dataset.iloc[:,-1]=='Iris-virginica', dataset.columns[-1]] = 2

## **2. 데이터 전처리(Data Preprocessing)**

In [3]:
train_X, test_X, train_y, test_y = train_test_split(dataset[dataset.columns[0:4]].values, dataset.iloc[:,-1].values, test_size=0.2)

In [4]:
# Pytorch에서 수학적 연산을 가속화하기 위해 tensor data를 사용한다.

train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())

train_y = np.array(train_y, dtype=np.int64)
test_y = np.array(test_y, dtype=np.int64)
train_y = torch.from_numpy(train_y)
test_y = torch.from_numpy(test_y)

## 3. 신경망 구축(Building a Neural Network)

In [5]:
class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)

        return X    

In [6]:
# Net 클래스를 인스턴스화하고 model 객체를 생성합니다.
# torch.nn.Module의 하위 클래스는 생성한 레이어와 해당 모양 및 매개변수를 보고합니다.

model = Net()
print(model)

Net(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)


### 4. Optimizer 설정

In [7]:
# loss function: 모델의 예측이 이상적인 출력에서 얼마나 멀리 떨어져 있는지 측정
# optimizer: 학습을 주도(확률적 경사 하강법)
#            학습률, 운동량 외에도 모든 학습 가중치 모음인 net.parameters()도 전달

criterion = nn.CrossEntropyLoss()# cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### 5. 학습 모델 훈련(Train Model)

In [8]:
# loss function과 optimizer를 훈련 루프에 통합

epochs = 100
for epoch in range(1, epochs+1):
    optimizer.zero_grad()
    output = model(train_X)
    loss = criterion(output, train_y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print('number of epoch', epoch, 'loss', loss.data)

number of epoch 10 loss tensor(0.8627)
number of epoch 20 loss tensor(0.6934)
number of epoch 30 loss tensor(0.5987)
number of epoch 40 loss tensor(0.5811)
number of epoch 50 loss tensor(0.5796)
number of epoch 60 loss tensor(0.5749)
number of epoch 70 loss tensor(0.5723)
number of epoch 80 loss tensor(0.5705)
number of epoch 90 loss tensor(0.5691)
number of epoch 100 loss tensor(0.5682)


### 6. 추론(Inference)

In [9]:
# test_X를 model에 통과시켜 tensor(Iris-setosa, Iris-versicolor, Iris-virginica)로 이루어진 튜플 predict_out를 출력한다.
# torch.max() 함수를 통해, predict_out의 튜플들 중에서 가장 높은 숫자로 이루어진 1차원 tensor 배열 predict_y를 출력한다.

predict_out = model(test_X)
_, predict_y = torch.max(predict_out, 1)

In [10]:
# tensor(0, 1, 2)를 Iris-setosa, Iris-versicolor, Iris-virginica로 변환해준다.
# 예상값들을 리스트 prediction에 넣어 저장한다.
prediction = []

for data in predict_y:
    if data == 0:
        prediction.append("Iris-setosa")
    elif data == 1:
        prediction.append("Iris-versicolor")
    else:
        prediction.append("Iris-virginica")

In [11]:
# 실제값(test_y)과 예상값(predict_y)의 Accuracy를 계산하고 prediction에 저장된 리스트를 출력한다.

print('prediction accuracy: \t', accuracy_score(test_y.data, predict_y.data))
print(prediction)

prediction accuracy: 	 0.9666666666666667
['Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica']
